# 1. Installing libraries 

In [ ]:
# !pip install neuralforecast datasetsforecast;

# 2. Load ETTm2 Data

In [ ]:
import pandas as pd
from datasetsforecast.long_horizon import LongHorizon

C:\Users\dhavy\AppData\Local\Temp\ipykernel_30528\156242825.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was too old on your system - pyarrow 10.0.1 is the current minimum supported version as of this release.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [ ]:
# Change this to your own data to try the model
Y_df, _, _ = LongHorizon.load(directory='./', group='ETTm2')
Y_df['ds'] = pd.to_datetime(Y_df['ds'])

n_time = len(Y_df.ds.unique())
val_size = int(.2 * n_time)
test_size = int(.2 * n_time)

Y_df.groupby('unique_id').head(2)
Y_df.head()

,unique_id,ds,y
0,HUFL,2016-07-01 00:00:00,-0.041413
1,HUFL,2016-07-01 00:15:00,-0.185467
2,HUFL,2016-07-01 00:30:00,-0.257495
3,HUFL,2016-07-01 00:45:00,-0.577510
4,HUFL,2016-07-01 01:00:00,-0.385501


# 3. Train models

In [ ]:
import numpy as np
import pandas as pd
import pytorch_lightning as pl
import matplotlib.pyplot as plt

from neuralforecast import NeuralForecast
from neuralforecast.losses.pytorch import MAE
from neuralforecast.core import NeuralForecast
from neuralforecast.models import Informer, Autoformer, FEDformer, PatchTST, TSMixer

C:\Sapna\Nixtla_Project\neuralforecast\neuralforecast\utils.py:281: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  AirPassengersPanel["y_[lag12]"].fillna(AirPassengersPanel["y"], inplace=True)


In [ ]:
models = [TSMixer(h=12,
                input_size=12,
                n_series=2,
                # stat_exog_list=['airline1'],
                # futr_exog_list=['trend'],
                scaler_type='robust',
                max_steps=200,
                early_stop_patience_steps=-1,
                val_check_steps=10,
                learning_rate=1e-3,
                loss=MAE(),
                valid_loss=None,
                batch_size=12,
                n_block=1,
                ff_dim=64,
                target_slice=None
                )
         ]

INFO:lightning_fabric.utilities.seed:Seed set to 1


In [ ]:
nf = NeuralForecast(
    models=models,
    freq='ME')

In [ ]:
Y_hat_df = nf.cross_validation(df=Y_df,
                               val_size=val_size,
                               test_size=test_size,
                               n_windows=None
                              )

Sanity Checking: |                                                                               | 0/? [00:00<…

In [ ]:
# Y_hat_df.head()

# 4. Evaluate Results

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
Y_plot = Y_hat_df[Y_hat_df['unique_id']=='OT'] # OT dataset
cutoffs = Y_hat_df['cutoff'].unique()[::horizon]
Y_plot = Y_plot[Y_hat_df['cutoff'].isin(cutoffs)]

plt.figure(figsize=(20,5))
plt.plot(Y_plot['ds'], Y_plot['y'], label='True')
# plt.plot(Y_plot['ds'], Y_plot['Informer'], label='Informer')
# plt.plot(Y_plot['ds'], Y_plot['Autoformer'], label='Autoformer')
# plt.plot(Y_plot['ds'], Y_plot['PatchTST'], label='PatchTST')
plt.plot(Y_plot['ds'], Y_plot['TSMixer'], label='TSMixer')
plt.xlabel('Datestamp')
plt.ylabel('OT')
plt.grid()
plt.legend()

In [ ]:
from neuralforecast.losses.numpy import mae

In [ ]:
# mae_informer = mae(Y_hat_df['y'], Y_hat_df['Informer'])
# mae_autoformer = mae(Y_hat_df['y'], Y_hat_df['Autoformer'])
# mae_patchtst = mae(Y_hat_df['y'], Y_hat_df['PatchTST'])
mae_tsmixer = mae(Y_hat_df['y'], Y_hat_df['TSMixer'])

# print(f'Informer: {mae_informer:.3f}')
# print(f'Autoformer: {mae_autoformer:.3f}')
# print(f'PatchTST: {mae_patchtst:.3f}')
print(f'TSMixer: {mae_tsmixer:.3f}')